In [1]:
# 감성어 사전: 한국어 감성어 사전 - KNU 감성어 사전

### +

In [6]:
words = ['바다', '비', '비행기', '드라이브', '기차', '산', '계곡', '카페', '페스티벌', '공원']

In [3]:
# naver api

# import os
# import sys
# import urllib.request
# client_id = 'id'
# client_secret = 'seceret'
# encText = urllib.parse.quote(word)
# # url = 'https://openapi.naver.com/v1/search/blog?query=' + encText # JSON 결과
# url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # XML 결과
# request = urllib.request.Request(url)
# request.add_header("X-Naver-Client-Id",client_id)
# request.add_header("X-Naver-Client-Secret",client_secret)
# response = urllib.request.urlopen(request)
# rescode = response.getcode()
# if(rescode==200):
#     response_body = response.read()
#     print(response_body.decode('utf-8'))
# else:
#     print("Error Code:" + rescode)

In [1]:
import pandas as pd
import time
import re

from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import requests
from bs4 import BeautifulSoup

from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By

from konlpy.tag import * ; okt = Okt()

In [2]:
def get_url(n, keyword):
    urls = []
    url_info = f'https://search.naver.com/search.naver?query={keyword}&nso=&where=blog&sm=tab_opt'

    res = requests.get(url_info)
    soup = BeautifulSoup(res.content, 'html.parser')

    urls_info = soup.select('li.bx > div > div.total_area > a')

    for i in range(0, n):
        urls.append(urls_info[i]['href'])

    urls = [url for url in urls if 'naver' in url]
    return urls

In [3]:
def get_words(urls):
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.implicitly_wait(3)

    contents = []
    for url in urls:

        driver.get(url)
        time.sleep(1)

        driver.switch_to.frame("mainFrame")

        try:
            content = driver.find_element(By.CSS_SELECTOR,'div.se-main-container').text
            contents.append(content)

        except NoSuchElementException:
            content = driver.find_element(By.CSS_SELECTOR,'div#content-area').text
            contents.append(content)
        #print(본문: '\n', content)

    driver.quit()

    return contents

In [4]:
def make_df(n, words):
    word_content =[]
    today = time.strftime('%Y%m%d')
    
    for keyword in words:
        urls = get_url(n, keyword)
        content = get_words(urls)
        content = ' '.join(content)

        content = re.sub('[.!]', ' ', content)
        content = re.sub('[^가-힣0-9a-zA-Z\\s]', '', content)
        content = re.sub(' +', ' ', content)

        content = okt.morphs(content)
        content = [notoneword for notoneword in content if not len(notoneword) == 1]
    
        word_content.append(content)
    
    
    category_words = pd.DataFrame({'테마':words, '단어':word_content})
    category_words.to_csv(f'./data/CategoryWord_info_{today}.csv',encoding='utf-8-sig',index=False)

In [10]:
make_df(30,words)

[WDM] - Downloading: 100%|█████████████████| 8.15M/8.15M [00:00<00:00, 61.1MB/s]
/var/folders/mt/cl5sbk5935d2kz0_4_5kxkkr0000gn/T/ipykernel_90171/1564478895.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


### ++

In [11]:
import numpy as np
import json
import re
import gensim

from konlpy.tag import * ; okt = Okt()

In [12]:
class diary:
    def __init__(self, text):
        self.text = text
        self.stop = 0
        self.word_scores = []

    def prep(self):
        self.text = re.sub('[.!]', ' ', self.text)
        self.test = re.sub('[^가-힣0-9a-zA-Z\\s]', '', self.text)
        self.text = re.sub(' +', ' ', self.text)

        return self.text

    def tokenizer(self):
        self.word_list = okt.morphs(self.text)
        self.word_list = [notoneword for notoneword in self.word_list if not len(notoneword) == 1]
        
        return self.word_list
    
    def get_score(self):
        with open('data/SentiWord_info.json', encoding='utf-8-sig', mode='r') as f:
            data = json.load(f)
        score = 0

        for word in self.word_list:        
            for i in range(0, len(data)):
                if data[i]['word'] == word:
                    s_word = data[i]['polarity']
                    self.word_scores.append(int(s_word))
                
        return self.word_scores

    def get_ratio(self):
        ent_score = 0
        
        ent_score = np.sum(self.word_scores)
        length = np.log10(len(self.word_scores))+1

        try:
            self.ratio= round(ent_score/length,2)
        except:
            self.ratio = 0

        return self.ratio
    
    
#     def update_dict(self):
        

In [26]:
class result:
    def __init__(self, text):
        self.text = text
        res = diary(self.text)
        res.prep()
        res.prep()
        res.tokenizer()
        token = res.tokenizer()
        res.get_score()
        ratio = res.get_ratio()
        
        if ratio < -1: feel = 'Dark'
        elif ratio >1: feel = 'Bright'
        else: feel = 'Mid'
        
        # 임시 카운터
        words = ['바다', '비', '비행기', '드라이브', '기차', '산', '계곡', '카페', '페스티벌', '공원']
        count = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        
        for i in range(0, len(words)):
            for j in range(0, len(token)):
                if words[i] in token[j]: count[i] += 1
        
        try: noise = words[np.argmax(count)]
        except: noise = 'nomal'
        
        print('\n', ratio, '의 정도로 ', feel, '이다.', '\n', '그리고 noise는', noise, '이다.')
        
#         return ratio, feel, noise

In [27]:
result('오늘은 부산에 여행온지 2일차가 되었다. 친구들과 남포동에 택시를 타고 갔더니 어제 마신 술이 올라오는 것 같아서 어지러웠다. 그래도 도착해서 점심으로 먹은 피자와 파스타는 엄청 맛있었다. 오후에는 시장 구경을 다니다가 잠시 쉬기 위해 만화카페에 들어갔다. 만화카페에 고양이가 있었는데 너무 귀여워서 사진을 찍었다. 고양이는 앞으로 세상을 구할 것이다. 저녁시간이 다 되어서 만화카페를 나왔을 때 비가 내리기 시작했다. 아무래도 장마철에 여행을 와서 그런지 장대비가 쏟아져 내렸다. 한 여름이였지만 날이 너무 시원하고 옷은 축축했다. 기분이 좋은데 나빴달까. 이 후에 저녁 식사를 위해 삼겹살 집에 다녀왔다. 가게에서는 특이한 술잔을 사용하고 있어서 신기했다. 낮에 숙취를 겪었어도 여행 내내 술이 빠질 수는 없다. 이런저런 일들이 많았지만 참 즐거운 하루였다!')


 2.5 의 정도로  Bright 이다. 
 그리고 noise는 카페 이다.
